**Firstly, you should download the model file "pytorch-openpose-master" to your local system, and then modify the relevant path information to ensure the code runs successfully.**

you can find the master model file and the processed videos in  my Google Drive :

**https://drive.google.com/drive/folders/1CgMIg8yLFkGJvWBV_nZk1m35ETliu7Ya?usp=drive_link**

This code snippet imports the `sys` module and appends a specified path to the system path. The path `/Users/wangjiji/Desktop/pytorch-openpose-master/model` should be adjusted to the location of the 'src' directory within the downloaded "pytorch-openpose-master" folder on your local system. This ensures that the Python interpreter can locate and import modules from this directory.

In [1]:
import sys
sys.path.append('/Users/wangjiji/Desktop/pytorch-openpose-master/model')  # Adjust this path to the root where 'src' is located in the downloaded "pytorch-openpose-master" folder


Testing Codecs with OpenCV

To find out which codecs work with OpenCV on your system, you can experiment by trying to write a small video using different four-character codec identifiers. Here’s a simple Python script that attempts to write a short video with a variety of common codecs to see which ones produce a playable file:

In [ ]:
import cv2
import numpy as np

width, height = 640, 480
fps = 24
frame_count = 60

codecs = ['DIVX', 'XVID', 'MJPG', 'X264', 'mp4v', 'avc1']
for codec in codecs:
    video_filename = f'test_{codec.lower()}.avi' if codec != 'mp4v' and codec != 'avc1' else f'test_{codec.lower()}.mp4'
    print(f"Testing codec: {codec} in file {video_filename}")
    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(video_filename, fourcc, fps, (width, height))
    
    for _ in range(frame_count):
        frame = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)
        out.write(frame)
    
    out.release()

    if out.isOpened():
        print(f"Codec {codec} seems to work! Saved to {video_filename}")
    else:
        print(f"Codec {codec} did not work.")

Testing libx264 with OpenCV

Since OpenCV’s VideoWriter might not directly list libx264 due to licensing and build options, it usually supports avc1 or mp4v for H.264 encoding on most systems. You can test both.

Here’s how you can modify the script to use avc1 for better compatibility on macOS, where this codec is generally supported out of the box by the system’s video infrastructure:

In [4]:
import cv2
import numpy as np
import os

def test_video_writer(codec, file_extension):
    width, height = 640, 480
    fps = 30
    frame_count = 60

    output_filename = f'test_video_{codec.lower()}.{file_extension}'
    fourcc = cv2.VideoWriter_fourcc(*codec)
    out = cv2.VideoWriter(output_filename, fourcc, fps, (width, height))

    for _ in range(frame_count):
        frame = np.random.randint(0, 256, (height, width, 3), dtype=np.uint8)
        
        out.write(frame)

    out.release()
    print(f"Generated video with codec {codec}: {output_filename}")

# Testing 'avc1' with MP4 container, commonly supported on macOS.
test_video_writer('avc1', 'mp4')

Generated video with codec avc1: test_video_avc1.mp4


If the message shows "Generated video with codec avc1: test_video_avc1.mp4", then you should confidently use this codec in your main video processing script to ensure compatibility and playability on macOS systems.


**TEST PART you can choose not to run following codes**

Simplify and Verify Basic Video Writing

First, let’s test if we can simply read from a video and write it back without any processing. This step helps to confirm that the basic video writing functionality works:

In [6]:
import cv2
import os

def test_video_copy(video_file, output_folder):
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Failed to open video file: {video_file}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'avc1')

    output_file = os.path.join(output_folder, 'output_video.mp4')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)

    cap.release()
    out.release()
    print("Video copy test complete. Check the output video.")

if __name__ == '__main__':
    video_file = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/001.mp4'
    output_folder = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed'
    os.makedirs(output_folder, exist_ok=True)
    test_video_copy(video_file, output_folder)

Video copy test complete. Check the output video.


**Step-by-Step Integration of OpenPose Models**

To reintegrate the OpenPose model processing without disrupting the functionality:

	1.	Ensure Models are Loaded Properly: Verify that the model paths are correct and that the models are being loaded without errors. This is crucial for the next steps to function correctly.
	2.	Add Basic OpenPose Processing: Begin with minimal modifications to the frame processing to ensure each addition works without issues.

**WARNING:the average time to process one video (lenth of 1 second)is over 90 seconds**

In [19]:
import cv2
import numpy as np
import os
from src.body import Body
from src import util

# Initialize the Body pose model
body_estimation = Body('/Users/wangjiji/Desktop/pytorch-openpose-master/model/body_pose_model.pth')

def process_frame(frame):
    # Process the frame using the Body object
    candidate, subset = body_estimation(frame)
    canvas = frame.copy()
    canvas = util.draw_bodypose(canvas, candidate, subset)

    return canvas

def main(video_file, output_folder):
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print("Cannot open video file")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_file = os.path.join(output_folder, 'processed_video.mp4')
    out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_frame(frame)
        out.write(processed_frame)

    cap.release()
    out.release()
    print("Processed video has been written.")

if __name__ == '__main__':
    video_file = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/001.mp4'  # Path to the video file
    output_folder = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed'  # Path to save the processed video
    os.makedirs(output_folder, exist_ok=True)
    main(video_file, output_folder)

Processed video has been written.


Now, You can check the first video processed with body movement estimation, for batch processing, pls use following code.

**you can choose the output file format "video" or "gif".**

**The overall processing time will be over 90 minutues, pls also check the uploaded processed video in github repo**

In [24]:
import cv2
import numpy as np
import os
from src.body import Body
from src import util
from tqdm import tqdm
from moviepy.editor import ImageSequenceClip

# Initialize the Body pose model
body_estimation = Body('/Users/wangjiji/Desktop/pytorch-openpose-master/model/body_pose_model.pth')

def process_frame(frame):
    # Process the frame using the Body object
    candidate, subset = body_estimation(frame)
    canvas = frame.copy()
    canvas = util.draw_bodypose(canvas, candidate, subset)

    return canvas

def process_video(video_file, output_file, output_type='video'):
    cap = cv2.VideoCapture(video_file)
    if not cap.isOpened():
        print(f"Cannot open video file {video_file}")
        return

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    if output_type == 'video':
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_file, fourcc, fps, (width, height))
    else:
        frames = []

    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    progress_bar = tqdm(total=frame_count, desc=f"Processing {os.path.basename(video_file)}", leave=False)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        processed_frame = process_frame(frame)
        if output_type == 'video':
            out.write(processed_frame)
        else:
            frames.append(processed_frame)
        progress_bar.update(1)

    cap.release()
    if output_type == 'video':
        out.release()
    else:
        clip = ImageSequenceClip([cv2.cvtColor(f, cv2.COLOR_BGR2RGB) for f in frames], fps=30)
        clip.write_gif(output_file)

    progress_bar.close()
    print(f"Processed video has been written to {output_file}")

def main(input_folder, output_folder, output_type='video'):
    os.makedirs(output_folder, exist_ok=True)
    video_files = [os.path.join(input_folder, f"{i:03}.mp4") for i in range(1, 72)]
    valid_video_files = [f for f in video_files if os.path.exists(f)]
    
    total_videos = len(valid_video_files)
    overall_progress = tqdm(total=total_videos, desc="Overall Progress: 0% completed", unit='video')

    for i, video_file in enumerate(valid_video_files, start=1):
        if output_type == 'video':
            output_file = os.path.join(output_folder, f"{os.path.basename(video_file).replace('.mp4', '.mp4')}")
        else:
            output_file = os.path.join(output_folder, f"{os.path.basename(video_file).replace('.mp4', '.gif')}")

        process_video(video_file, output_file, output_type)
        percentage_completed = (i / total_videos) * 100
        overall_progress.set_description(f"Overall Progress: {percentage_completed:.2f}% completed")
        overall_progress.update(1)
    
    overall_progress.close()

if __name__ == '__main__':
    input_folder = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4'  # Path to the input folder
    output_folder = '/Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed'  # Path to save the processed videos
    os.makedirs(output_folder, exist_ok=True)

    user_input = input("Do you want to generate video or gif? (enter 'video' or 'gif'): ").strip().lower()
    if user_input in ['video', 'gif']:
        output_type = user_input
    else:
        output_type = 'video'  # Default to video if input is invalid

    main(input_folder, output_folder, output_type)

Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/001.mp4


Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/002.mp4


Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/003.mp4


Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/004.mp4


Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/005.mp4


...

Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/068.mp4


Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/069.mp4


Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/070.mp4








































































































































































Overall Progress: 100.00% completed: 100%|██████████| 59/59 [1:38:34<00:00, 100.24s/video]

Processed video has been written to /Users/wangjiji/Desktop/Social touch deep learning project/STMP4/Openpose processed/071.mp4
